In [1]:
import datetime
import json
import logging
import pprint
import time

import ee
from IPython.display import Image

import openet.ssebop as ssebop
import utils

ee.Initialize()

logging.basicConfig(level=logging.INFO, format='%(message)s')

#### Inputs

In [29]:
start_date = '2018-10-01'
end_date = '2018-12-04'

# start_date = '2015-07-01'
# end_date = '2015-07-31'
# start_date = '2016-07-01'
# end_date = '2016-07-31'
# start_date = '2017-02-07'
# end_date = '2017-02-07'

overwrite_flag = True

min_pixel_count = 1000
# min_scene_count = 10
max_cloud_cover = 70
# These don't do anything yet...
# landsat5_flag = False
# landsat7_flag = True
# landsat8_flag = True

#### Tcorr Output Image Collection

In [20]:
tcorr_img_name = 'topowx_median_v0_daily'
tcorr_img_coll_id = 'projects/usgs-ssebop/tcorr_image/{}'.format(tcorr_img_name)
tcorr_img_coll = ee.ImageCollection(tcorr_img_coll_id);

#### Study Area

In [31]:
export_crs = 'EPSG:4326'
export_geom = ee.Geometry.Rectangle([-125, 24, -65, 50], proj=export_crs, geodesic=False)  # CONUS
# export_geom = ee.Geometry.Rectangle([-124, 35, -119, 42], proj=export_crs, geodesic=False)  # California
export_region = export_geom.bounds(1, export_crs).coordinates().getInfo()[0][:-1]

Refreshing due to a 401 (attempt 1/2)
Refreshing access_token


#### Tmax Collection

In [5]:
tmax_source = 'topowx'
tmax_version = 'median_v0'
tmax_name = '{}_{}'.format(tmax_source.lower(), tmax_version.lower())
tmax_coll_id = 'projects/usgs-ssebop/tmax/{}'.format(tmax_name)

tmax_coll = ee.ImageCollection(tmax_coll_id)
tmax_img = ee.Image(tmax_coll.first()).set('TMAX_VERSION', tmax_version.upper())

# print(ee.Image(tmax_median_coll.first()).projection().getInfo()['transform'])
# print(ee.Image(tmax_median_coll.first()).projection().getInfo()['crs'])
# print(ee.Image(tmax_median_coll.first()).getInfo()['bands'][0]['dimensions'])
tmax_geo = [0.00833333329998709, 0.0, -125.00416722008521, 0.0, -0.00833333329998709, 51.19583312184854]
tmax_crs = 'EPSG:4326'
tmax_shape = [7000, 3250]
tmax_extent = [tmax_geo[2], tmax_geo[5] + tmax_shape[1] * tmax_geo[4], 
               tmax_geo[2] + tmax_shape[0] * tmax_geo[0], tmax_geo[5]]

# Image(url=tmax_img.getThumbURL({'min': 270, 'max': 330, 'region': export_region}))
# embed=True, format='png'

#### Export Extent, Shape, Geo

In [6]:
# export_cs = 0.008333333333333333333333  # ~800m
# export_cs = 0.016666666666666666666666  # ~1600m
export_cs = 0.033333333333333333333333  # ~3200m
export_crs = 'EPSG:4326'

# Compute clipped Tmax grid (this is a disaster of code)
export_xy = ee.Array(export_geom.bounds(1, export_crs).coordinates().get(0)).transpose().toList()
export_xmin = ee.Number(ee.List(export_xy.get(0)).reduce(ee.Reducer.min()))
export_ymin = ee.Number(ee.List(export_xy.get(1)).reduce(ee.Reducer.min()))
export_xmax = ee.Number(ee.List(export_xy.get(0)).reduce(ee.Reducer.max()))
export_ymax = ee.Number(ee.List(export_xy.get(1)).reduce(ee.Reducer.max()))
# Snap to Tmax grid
export_xmin = export_xmin.subtract(tmax_extent[0]).divide(export_cs).floor().multiply(export_cs).add(tmax_extent[0])
export_ymin = export_ymin.subtract(tmax_extent[3]).divide(export_cs).floor().multiply(export_cs).add(tmax_extent[3])
export_xmax = export_xmax.subtract(tmax_extent[0]).divide(export_cs).ceil().multiply(export_cs).add(tmax_extent[0])
export_ymax = export_ymax.subtract(tmax_extent[3]).divide(export_cs).ceil().multiply(export_cs).add(tmax_extent[3])
#  Limit to Tmax grid
export_xmin = export_xmin.max(tmax_extent[0]).min(tmax_extent[2])
export_ymin = export_ymin.max(tmax_extent[1]).min(tmax_extent[3])
export_xmax = export_xmax.min(tmax_extent[0]).max(tmax_extent[2])
export_ymax = export_ymax.min(tmax_extent[1]).max(tmax_extent[3])

# export_extent = ee.List([export_xmin, export_ymin, export_xmax, export_ymax])
export_geo = [export_cs, 0.0, export_xmin, 0.0, -export_cs, export_ymax])
export_shape = [
  export_xmax.subtract(export_xmin).abs().divide(export_cs).int(),
  export_ymax.subtract(export_ymin).abs().divide(export_cs).int()]
print(export_shape)
print(export_geo)

[1749, 786]
[0.03333333333333333, 0.0, -125.00416722008521, 0.0, -0.03333333333333333, 51.19583312184854]


#### Export the Tcorr Image for each date

In [30]:
# Get current task list
tasks = utils.get_ee_tasks()
if logging.getLogger().getEffectiveLevel() == logging.DEBUG:
    logging.debug('  Tasks: {}'.format(len(tasks)))

# # Get current asset list
# asset_list = utils.get_ee_assets(tcorr_img_coll_id, shell_flag=True)
# logging.debug('Displaying first 10 images in collection')
# logging.debug(asset_list[:10])
    
start_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

for export_dt in utils.date_range(start_dt, end_dt, days=1, skip_leap_days=False):   
    logging.info('{}'.format(export_dt.strftime('%Y-%m-%d')))
    
    if export_dt > datetime.datetime.today():
        logging.info('  Unsupported date, skipping')
        continue
    
    task_id = 'tcorr_image_{}_{}'.format(tcorr_img_name, export_dt.strftime('%Y%m%d'))
    asset_id = '{}/{}'.format(tcorr_img_coll_id, export_dt.strftime('%Y%m%d'))
    logging.debug('  Task ID: {}'.format(task_id))
    logging.debug('  Asset ID: {}'.format(asset_id))
    
    if overwrite_flag:
        if task_id in tasks.keys():
            logging.info('  Task already submitted, cancelling')
            ee.data.cancelTask(tasks[task_id])
        # This is intentionally not an "elif" so that a task can be
        # cancelled and an existing image/file/asset can be removed
        # if asset_id in asset_list:
        if utils.image_exists(asset_id):
            logging.info('  Asset already exists, removing')
            ee.data.deleteAsset(asset_id)
    else:
        if task_id in tasks.keys():
            logging.info('  Task already submitted, skipping')
            continue
        # elif asset_id in asset_list:
        elif utils.image_exists(asset_id):
            logging.info('  Asset already exists, skipping')
            continue

    # Build and merge the Landsat collections
    l8_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA')\
        .filterDate(export_dt, export_dt + datetime.timedelta(days=1))\
        .filterBounds(tmax_img.geometry())\
        .filterBounds(export_geom)\
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', max_cloud_cover)\
        .filterMetadata('DATA_TYPE', 'equals', 'L1TP')
    l7_coll = ee.ImageCollection('LANDSAT/LE07/C01/T1_RT_TOA')\
        .filterDate(export_dt, export_dt + datetime.timedelta(days=1))\
        .filterBounds(tmax_img.geometry())\
        .filterBounds(export_geom)\
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', max_cloud_cover)\
        .filterMetadata('DATA_TYPE', 'equals', 'L1TP')
    landsat_coll = l8_coll.merge(l7_coll)
    # l5_coll = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')\
    #     .filterDate(export_dt, export_dt + datetime.timedelta(days=1))\
    #     .filterBounds(tmax_img.geometry())\
    #     .filterBounds(export_geom)\
    #     .filterMetadata('CLOUD_COVER_LAND', 'less_than', max_cloud_cover)\
    #     .filterMetadata('DATA_TYPE', 'equals', 'L1TP')
    # landsat_coll = l8_coll.merge(l7_coll).merge(l5_coll)
    # pprint.pprint(landsat_coll.aggregate_histogram('system:index').getInfo())
            
    def tcorr_img_func(image):
        t_stats = ssebop.Image.from_landsat_c1_toa(ee.Image(image)).tcorr_stats
        tcorr = ee.Algorithms.If(t_stats.get('tcorr_p5'), ee.Number(t_stats.get('tcorr_p5')), 0)
        # tcorr = ee.Number(t_stats.get('tcorr_p5'))
        count = ee.Number(t_stats.get('tcorr_count'))
        
        # Remove the merged collection indices from the system:index
        scene_id = ee.List(ee.String(image.get('system:index')).split('_')).slice(-3)
        scene_id = ee.String(scene_id.get(0)).cat('_')\
            .cat(ee.String(scene_id.get(1))).cat('_')\
            .cat(ee.String(scene_id.get(2)))
        
        return ee.Image([
                tmax_img.select([0], ['tcorr']).double().multiply(0).add(ee.Image.constant(tcorr)),
                tmax_img.select([0], ['count']).double().multiply(0).add(ee.Image.constant(count))])\
            .clip(image.geometry())\
            .updateMask(1)\
            .setMulti({
                # 'system:index': scene_id,
                'system:time_start': image.get('system:time_start'),
                'SCENE_ID': scene_id,
                'WRS2_TILE': scene_id.slice(5, 11),
                # 'WRS2_TILE': ee.String('p').cat(scene_id.slice(5, 8)).cat('r').cat(scene_id.slice(8, 11)),
                'TCORR': tcorr,
                'COUNT': count,
        })
        #     .copyProperties(image, ['system:time_start', 'system:index'])    
    tcorr_img_coll = ee.ImageCollection(landsat_coll.map(tcorr_img_func)) \
        .filterMetadata('COUNT', 'not_less_than', min_pixel_count)
    # pprint.pprint(tcorr_img_coll.aggregate_histogram('system:index').getInfo())
    # pprint.pprint(ee.Image(tcorr_img_coll.first()).getInfo())

    # DEADBEEF - This doesn't work since there seems to be a limit on the type and 
    #   length of properties for exported assets.
    # # Save Tcorr properties for the scene images as a property on the daily image
    # def tcorr_ftr_func(tcorr_img):
    #     return ee.Feature(
    #         None,
    #         {
    #             'SCENE_ID': ee.String(tcorr_img.get('SCENE_ID')),
    #             'TCORR': ee.Number(tcorr_img.get('TCORR')),
    #             'COUNT': ee.Number(tcorr_img.get('COUNT')),
    #             # 'SSEBOP_VER': ssebop.__version__,
    #             # 'TMAX_SOURCE': tmax_source.upper(),
    #             # 'TMAX_VERSION': tmax_version.upper(),
    #             # 'EXPORT_DATE': datetime.datetime.today().strftime('%Y-%m-%d'),
    #             # 'WRS2_TILE': ee.String('p').cat(scene_id.slice(5, 8)).cat('r').cat(scene_id.slice(8, 11)),
    #             # 'system:time_start': tcorr_img.get('system:time_start'),
    #             # 'system:index': tcorr_img.get('SCENE_ID'),
    #         })
    
    tcorr_img = tcorr_img_coll.mean()\
        .setMulti({
            'system:time_start': utils.millis(export_dt),
            'WRS2_TILES': ee.String(ee.List(ee.Dictionary(
                tcorr_img_coll.aggregate_histogram('WRS2_TILE')).keys()).join(',')),
            'SSEBOP_VERSION': ssebop.__version__,
            'TMAX_SOURCE': tmax_source.upper(),
            'TMAX_VERSION': tmax_version.upper(),
            'EXPORT_DATE': datetime.datetime.today().strftime('%Y-%m-%d'),
        })
    # pprint.pprint(tcorr_img.getInfo())

    task = ee.batch.Export.image.toAsset(
        image=tcorr_img,
        description=task_id,
        assetId=asset_id,
        crs=export_crs,
        crsTransform='[' + ','.join(list(map(str, export_geo))) + ']',
        dimensions='{0}x{1}'.format(*export_shape),
    )
    task.start()
    time.sleep(1)
    logging.debug('  Status: {}'.format(task.status()['state']))
    logging.debug('')

2018-10-01
  Asset already exists, removing
2018-10-02
  Asset already exists, removing
2018-10-03
  Asset already exists, removing
2018-10-04
  Asset already exists, removing
2018-10-05
  Asset already exists, removing
2018-10-06
  Asset already exists, removing
2018-10-07
  Asset already exists, removing
2018-10-08
  Asset already exists, removing
2018-10-09
  Asset already exists, removing
2018-10-10
  Asset already exists, removing
2018-10-11
  Asset already exists, removing
2018-10-12
  Asset already exists, removing
2018-10-13
  Asset already exists, removing
2018-10-14
  Asset already exists, removing
2018-10-15
  Asset already exists, removing
2018-10-16
  Asset already exists, removing
2018-10-17
  Asset already exists, removing
2018-10-18
  Asset already exists, removing
2018-10-19
  Asset already exists, removing
2018-10-20
  Asset already exists, removing
2018-10-21
  Asset already exists, removing
2018-10-22
  Asset already exists, removing
2018-10-23
  Asset already exist

In [ ]:
# tcorr_img = ee.Image('{}/{}'.format(tcorr_img_coll_id, '20170701'))
# Image(url=ee.Image(tcorr_img).getThumbURL({
#     'min': 0.95, 'max': 1.0, 'region': export_region,
#     'palette': ['ff0000', 'ffff00', '00ffff', '0000ff']}))
# # embed=True, format='png'